<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/MetaIntelligence_CLI_(utf_8)_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
MetaIntelligence CLI: train/eval/predict/benchmark/export-onnx

End-to-end, Colab/Notebook-safe launcher:
- Train an MLP on synthetic datasets (moons, circles, blobs)
- Save best checkpoint, scaler, config, and metrics
- Evaluate on test split (reproducible via saved config/seed)
- Predict on CSV with saved scaler/model (auto or explicit feature selection)
- Export ONNX graph
- Benchmark across multiple seeds

Dependencies:
  pip install torch scikit-learn pandas numpy joblib

Optional for convenience (exporting ONNX doesn't require onnx package):
  pip install onnx onnxruntime

"""

import os
import json
import time
import math
import shutil
import random
import argparse
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import List, Tuple, Optional, Dict, Any

import numpy as np
import pandas as pd
from joblib import dump as joblib_dump, load as joblib_load

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.datasets import make_moons, make_circles, make_blobs


# ------------------------------
# Utilities
# ------------------------------

def log(msg: str) -> None:
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

def now_ts() -> str:
    return time.strftime("%Y%m%d-%H%M%S")

def ensure_dir(path: str | Path) -> Path:
    p = Path(path)
    p.mkdir(parents=True, exist_ok=True)
    return p

def save_json(obj: Dict[str, Any], path: str | Path) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, sort_keys=True)

def load_json(path: str | Path) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def seed_everything(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # type: ignore[attr-defined]
    torch.backends.cudnn.benchmark = False     # type: ignore[attr-defined]

def sanitize_argv(raw_args: List[str]) -> List[str]:
    """
    Remove notebook-injected args (e.g., '-f', '...kernel-XXXX.json') and any trailing .json.
    """
    filtered = []
    skip_next = False
    for i, a in enumerate(raw_args):
        if skip_next:
            skip_next = False
            continue
        if a == "-f":
            # Skip '-f' and the following json path (Jupyter/IPython)
            skip_next = True
            continue
        if a.endswith(".json"):
            # Skip any json path injection
            continue
        filtered.append(a)
    return filtered


# ------------------------------
# Data
# ------------------------------

@dataclass
class DataConfig:
    dataset: str = "moons"           # moons|circles|blobs
    n_samples: int = 2000
    noise: float = 0.2
    centers: int = 3                 # for blobs only
    cluster_std: float = 1.0         # for blobs only
    val_split: float = 0.2
    test_split: float = 0.2
    seed: int = 42

def make_dataset(cfg: DataConfig) -> Tuple[np.ndarray, np.ndarray]:
    if cfg.dataset == "moons":
        X, y = make_moons(n_samples=cfg.n_samples, noise=cfg.noise, random_state=cfg.seed)
    elif cfg.dataset == "circles":
        X, y = make_circles(n_samples=cfg.n_samples, noise=cfg.noise, factor=0.5, random_state=cfg.seed)
    elif cfg.dataset == "blobs":
        X, y = make_blobs(n_samples=cfg.n_samples, centers=cfg.centers, cluster_std=cfg.cluster_std, random_state=cfg.seed)
    else:
        raise ValueError(f"Unknown dataset: {cfg.dataset}")
    return X.astype(np.float32), y.astype(np.int64)

def stratified_split(X: np.ndarray, y: np.ndarray, val_split: float, test_split: float, seed: int) -> Dict[str, np.ndarray]:
    """
    Returns indices dict: {'train_idx','val_idx','test_idx'} with stratification.
    """
    rng = np.random.RandomState(seed)
    classes = np.unique(y)
    train_idx, val_idx, test_idx = [], [], []
    for c in classes:
        idx = np.where(y == c)[0]
        rng.shuffle(idx)
        n = len(idx)
        n_test = int(round(n * test_split))
        n_val = int(round(n * val_split))
        n_train = n - n_test - n_val
        # Guarantee minimum presence
        n_train = max(n_train, 1)
        rest = n - n_train
        n_val = min(n_val, rest)
        n_test = rest - n_val
        t_idx = idx[:n_train]
        v_idx = idx[n_train:n_train + n_val]
        te_idx = idx[n_train + n_val:]
        train_idx.extend(t_idx.tolist())
        val_idx.extend(v_idx.tolist())
        test_idx.extend(te_idx.tolist())
    rng.shuffle(train_idx)
    rng.shuffle(val_idx)
    rng.shuffle(test_idx)
    return {
        "train_idx": np.array(train_idx, dtype=np.int64),
        "val_idx": np.array(val_idx, dtype=np.int64),
        "test_idx": np.array(test_idx, dtype=np.int64),
    }


# ------------------------------
# Model
# ------------------------------

class MLP(nn.Module):
    def __init__(self, in_dim: int, hidden: List[int], out_dim: int):
        super().__init__()
        layers: List[nn.Module] = []
        prev = in_dim
        for h in hidden:
            layers += [nn.Linear(prev, h), nn.ReLU()]
            prev = h
        layers.append(nn.Linear(prev, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


# ------------------------------
# Training / Evaluation
# ------------------------------

@dataclass
class TrainConfig:
    run_dir: str = "runs/run"
    hidden_sizes: List[int] = None  # type: ignore[assignment]
    lr: float = 1e-3
    weight_decay: float = 0.0
    epochs: int = 50
    batch_size: int = 64
    device: str = "cpu"

    def __post_init__(self):
        if self.hidden_sizes is None:
            self.hidden_sizes = [64, 64]

def build_loaders(X: np.ndarray, y: np.ndarray, idx: Dict[str, np.ndarray], batch_size: int) -> Tuple[DataLoader, DataLoader, DataLoader]:
    def make_dl(idxs):
        ds = TensorDataset(torch.from_numpy(X[idxs]), torch.from_numpy(y[idxs]))
        return DataLoader(ds, batch_size=batch_size, shuffle=True)
    return make_dl(idx["train_idx"]), make_dl(idx["val_idx"]), make_dl(idx["test_idx"])

def train_model(data_cfg: DataConfig, train_cfg: TrainConfig) -> Dict[str, Any]:
    seed_everything(data_cfg.seed)

    # Prepare run dir
    rd = ensure_dir(train_cfg.run_dir)
    (rd / "artifacts").mkdir(exist_ok=True)

    # Data
    X, y = make_dataset(data_cfg)
    idx = stratified_split(X, y, data_cfg.val_split, data_cfg.test_split, data_cfg.seed)

    # Scaler (fit on train only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X[idx["train_idx"]])
    X_val = scaler.transform(X[idx["val_idx"]])
    X_test = scaler.transform(X[idx["test_idx"]])

    X_scaled = np.zeros_like(X)
    X_scaled[idx["train_idx"]] = X_train
    X_scaled[idx["val_idx"]] = X_val
    X_scaled[idx["test_idx"]] = X_test

    # Save artifacts for reproducibility
    cfg = {
        "data": asdict(data_cfg),
        "train": {
            **asdict(train_cfg),
            "hidden_sizes": train_cfg.hidden_sizes,
            "device": train_cfg.device,
        },
    }
    save_json(cfg, rd / "config.json")
    save_json({k: v.tolist() for k, v in idx.items()}, rd / "indices.json")
    joblib_dump(scaler, rd / "scaler.pkl")

    # Model
    in_dim = X.shape[1]
    out_dim = len(np.unique(y))
    model = MLP(in_dim, train_cfg.hidden_sizes, out_dim).to(train_cfg.device)
    opt = torch.optim.Adam(model.parameters(), lr=train_cfg.lr, weight_decay=train_cfg.weight_decay)
    criterion = nn.CrossEntropyLoss()

    train_loader, val_loader, test_loader = build_loaders(X_scaled, y, idx, train_cfg.batch_size)

    best_val = -math.inf
    history = []
    best_path = rd / "best.pt"

    for epoch in range(1, train_cfg.epochs + 1):
        model.train()
        train_losses = []
        train_y_true, train_y_pred = [], []

        for xb, yb in train_loader:
            xb = xb.to(train_cfg.device)
            yb = yb.to(train_cfg.device)
            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            opt.step()
            train_losses.append(loss.item())
            pred = logits.argmax(dim=1).detach().cpu().numpy()
            train_y_true.extend(yb.detach().cpu().numpy())
            train_y_pred.extend(pred)

        train_acc = accuracy_score(train_y_true, train_y_pred)
        train_loss = float(np.mean(train_losses))

        # Validation
        model.eval()
        val_y_true, val_y_pred, val_losses = [], [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(train_cfg.device)
                yb = yb.to(train_cfg.device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_losses.append(loss.item())
                pred = logits.argmax(dim=1).detach().cpu().numpy()
                val_y_true.extend(yb.detach().cpu().numpy())
                val_y_pred.extend(pred)

        val_acc = accuracy_score(val_y_true, val_y_pred)
        val_loss = float(np.mean(val_losses))

        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
        })

        log(f"Epoch {epoch:03d} | train_acc={train_acc:.3f} val_acc={val_acc:.3f} train_loss={train_loss:.4f} val_loss={val_loss:.4f}")

        # Track best by val_acc
        if val_acc > best_val:
            best_val = val_acc
            torch.save({"model_state": model.state_dict(), "in_dim": in_dim, "out_dim": out_dim, "hidden": train_cfg.hidden_sizes}, best_path)

    # Save history
    pd.DataFrame(history).to_csv(rd / "history.csv", index=False)
    metrics = {"best_val_acc": best_val}
    save_json(metrics, rd / "metrics.json")

    # Also evaluate on test split immediately (optional)
    test_metrics = evaluate_model(rd)
    save_json({"best_val_acc": best_val, **test_metrics}, rd / "metrics_final.json")

    return {"run_dir": str(rd), "best_val_acc": best_val, **test_metrics}


def evaluate_model(run_dir: str | Path) -> Dict[str, Any]:
    rd = Path(run_dir)
    cfg = load_json(rd / "config.json")
    idx = {k: np.array(v, dtype=np.int64) for k, v in load_json(rd / "indices.json").items()}
    scaler: StandardScaler = joblib_load(rd / "scaler.pkl")
    checkpoint = torch.load(rd / "best.pt", map_location="cpu")

    data_cfg = DataConfig(**cfg["data"])
    X, y = make_dataset(data_cfg)

    X_test = scaler.transform(X[idx["test_idx"]])
    y_test = y[idx["test_idx"]]

    model = MLP(checkpoint["in_dim"], checkpoint["hidden"], checkpoint["out_dim"])
    model.load_state_dict(checkpoint["model_state"])
    model.eval()

    with torch.no_grad():
        logits = model(torch.from_numpy(X_test))
        probs = torch.softmax(logits, dim=1).numpy()
        preds = probs.argmax(axis=1)

    acc = float(accuracy_score(y_test, preds))
    ll = float(log_loss(y_test, probs, labels=list(range(checkpoint["out_dim"]))))

    results = {"test_acc": acc, "test_log_loss": ll, "n_test": int(len(y_test))}
    log(f"Eval | test_acc={acc:.3f} test_log_loss={ll:.4f} n_test={len(y_test)}")
    save_json(results, rd / "eval.json")
    return results


def predict_csv(run_dir: str | Path, csv_path: str | Path, output: Optional[str] = None,
                features: Optional[List[str]] = None, include_proba: bool = True) -> str:
    rd = Path(run_dir)
    scaler: StandardScaler = joblib_load(rd / "scaler.pkl")
    checkpoint = torch.load(rd / "best.pt", map_location="cpu")

    model = MLP(checkpoint["in_dim"], checkpoint["hidden"], checkpoint["out_dim"])
    model.load_state_dict(checkpoint["model_state"])
    model.eval()

    df = pd.read_csv(csv_path)
    if features is None:
        # Auto-select numeric columns, excluding a column named 'label' if present
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        features = [c for c in num_cols if c.lower() != "label"]
    X = df[features].to_numpy(dtype=np.float32)

    # Expect same dimensionality as training
    if X.shape[1] != checkpoint["in_dim"]:
        raise ValueError(f"Feature dimension mismatch: model expects {checkpoint['in_dim']} but CSV has {X.shape[1]}")

    Xs = scaler.transform(X)
    with torch.no_grad():
        logits = model(torch.from_numpy(Xs))
        probs = torch.softmax(logits, dim=1).numpy()
        preds = probs.argmax(axis=1)

    out_df = df.copy()
    out_df["prediction"] = preds
    if include_proba:
        for k in range(probs.shape[1]):
            out_df[f"proba_{k}"] = probs[:, k]

    out_path = output or (str(Path(csv_path).with_suffix("")) + "_pred.csv")
    out_df.to_csv(out_path, index=False)
    log(f"Wrote predictions to: {out_path}")
    return out_path


def export_onnx(run_dir: str | Path, out_path: Optional[str] = None) -> str:
    rd = Path(run_dir)
    checkpoint = torch.load(rd / "best.pt", map_location="cpu")
    model = MLP(checkpoint["in_dim"], checkpoint["hidden"], checkpoint["out_dim"])
    model.load_state_dict(checkpoint["model_state"])
    model.eval()

    dummy = torch.randn(1, checkpoint["in_dim"], dtype=torch.float32)
    onnx_path = out_path or str(rd / "model.onnx")
    torch.onnx.export(
        model,
        dummy,
        onnx_path,
        input_names=["input"],
        output_names=["logits"],
        dynamic_axes={"input": {0: "batch"}, "logits": {0: "batch"}},
        opset_version=13,
    )
    # Save scaler params for reference
    scaler: StandardScaler = joblib_load(rd / "scaler.pkl")
    save_json({"mean": scaler.mean_.tolist(), "scale": scaler.scale_.tolist()}, rd / "scaler_params.json")
    log(f"Exported ONNX model to: {onnx_path}")
    return onnx_path


def benchmark(data_cfg: DataConfig, train_cfg: TrainConfig, seeds: List[int]) -> Dict[str, Any]:
    results = []
    base = Path(train_cfg.run_dir)
    base.mkdir(parents=True, exist_ok=True)
    for s in seeds:
        run_subdir = base / f"{data_cfg.dataset}_seed{s}_{now_ts()}"
        dc = DataConfig(**{**asdict(data_cfg), "seed": s})
        tc = TrainConfig(**{**asdict(train_cfg), "run_dir": str(run_subdir)})
        log(f"Benchmark seed={s}")
        out = train_model(dc, tc)
        results.append({"seed": s, **out})

    # Aggregate
    accs = [r["test_acc"] for r in results if "test_acc" in r]
    mean_acc = float(np.mean(accs)) if accs else float("nan")
    std_acc = float(np.std(accs)) if accs else float("nan")
    agg = {"mean_test_acc": mean_acc, "std_test_acc": std_acc, "runs": results}
    save_json(agg, base / "benchmark_results.json")
    log(f"Benchmark | mean_test_acc={mean_acc:.3f} std={std_acc:.3f} over {len(accs)} runs")
    return agg


# ------------------------------
# CLI
# ------------------------------

def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(
        prog="metaintel",
        description="MetaIntelligence CLI: train/eval/predict/benchmark/export-onnx",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    subparsers = parser.add_subparsers(dest="command")  # don't require to keep notebook-friendly

    # train
    p_train = subparsers.add_parser("train", help="Train a model", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_train.add_argument("--dataset", choices=["moons", "circles", "blobs"], default="moons")
    p_train.add_argument("--n_samples", type=int, default=2000)
    p_train.add_argument("--noise", type=float, default=0.2)
    p_train.add_argument("--centers", type=int, default=3, help="for blobs")
    p_train.add_argument("--cluster_std", type=float, default=1.0, help="for blobs")
    p_train.add_argument("--val_split", type=float, default=0.2)
    p_train.add_argument("--test_split", type=float, default=0.2)
    p_train.add_argument("--seed", type=int, default=42)

    p_train.add_argument("--hidden_sizes", type=str, default="64,64", help="Comma-separated hidden sizes")
    p_train.add_argument("--lr", type=float, default=1e-3)
    p_train.add_argument("--weight_decay", type=float, default=0.0)
    p_train.add_argument("--epochs", type=int, default=50)
    p_train.add_argument("--batch_size", type=int, default=64)
    p_train.add_argument("--device", type=str, default="cpu")
    p_train.add_argument("--run_dir", type=str, default=f"runs/{now_ts()}")

    # eval
    p_eval = subparsers.add_parser("eval", help="Evaluate best checkpoint on test set", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_eval.add_argument("--run_dir", type=str, required=True)

    # predict
    p_pred = subparsers.add_parser("predict", help="Predict on CSV (tabular only)", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_pred.add_argument("--run_dir", type=str, required=True)
    p_pred.add_argument("--csv", type=str, required=True)
    p_pred.add_argument("--output", type=str, default=None, help="Output CSV path")
    p_pred.add_argument("--features", type=str, default=None, help="Comma-separated feature columns (default: auto numeric cols)")
    p_pred.add_argument("--no-proba", action="store_true", help="Do not include probability columns")

    # export-onnx
    p_onnx = subparsers.add_parser("export-onnx", help="Export best checkpoint to ONNX", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_onnx.add_argument("--run_dir", type=str, required=True)
    p_onnx.add_argument("--output", type=str, default=None)

    # benchmark
    p_bench = subparsers.add_parser("benchmark", help="Benchmark multiple seeds", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    p_bench.add_argument("--dataset", choices=["moons", "circles", "blobs"], default="moons")
    p_bench.add_argument("--n_samples", type=int, default=2000)
    p_bench.add_argument("--noise", type=float, default=0.2)
    p_bench.add_argument("--centers", type=int, default=3)
    p_bench.add_argument("--cluster_std", type=float, default=1.0)
    p_bench.add_argument("--val_split", type=float, default=0.2)
    p_bench.add_argument("--test_split", type=float, default=0.2)
    p_bench.add_argument("--hidden_sizes", type=str, default="64,64")
    p_bench.add_argument("--lr", type=float, default=1e-3)
    p_bench.add_argument("--weight_decay", type=float, default=0.0)
    p_bench.add_argument("--epochs", type=int, default=50)
    p_bench.add_argument("--batch_size", type=int, default=64)
    p_bench.add_argument("--device", type=str, default="cpu")
    p_bench.add_argument("--run_dir", type=str, default=f"runs/bench_{now_ts()}")
    p_bench.add_argument("--seeds", type=str, default="1,2,3,4,5", help="Comma-separated seeds")

    return parser


# ------------------------------
# Commands
# ------------------------------

def cmd_train(args: argparse.Namespace) -> None:
    data_cfg = DataConfig(
        dataset=args.dataset,
        n_samples=args.nsamples,
        noise=args.noise,
        centers=args.centers,
        cluster_std=args.clusterstd,
        val_split=args.valsplit,
        test_split=args.testsplit,
        seed=args.seed,
    )
    hidden = [int(x) for x in args.hidden_sizes.split(",") if x.strip()]
    train_cfg = TrainConfig(
        run_dir=args.rundir,
        hidden_sizes=hidden,
        lr=args.lr,
        weight_decay=args.weightdecay,
        epochs=args.epochs,
        batch_size=args.batchsize,
        device=args.device,
    )
    log("Starting training...")
    out = train_model(data_cfg, train_cfg)
    log(f"Done. Run dir: {out['run_dir']}")


def cmd_eval(args: argparse.Namespace) -> None:
    log("Evaluating best checkpoint...")
    evaluate_model(args.rundir)


def cmd_predict(args: argparse.Namespace) -> None:
    feats = [f.strip() for f in args.features.split(",") if f.strip()] if args.features else None
    predict_csv(args.rundir, args.csv, output=args.output, features=feats, include_proba=not args.noproba)


def cmd_export_onnx(args: argparse.Namespace) -> None:
    export_onnx(args.rundir, args.output)


def cmd_benchmark(args: argparse.Namespace) -> None:
    data_cfg = DataConfig(
        dataset=args.dataset,
        n_samples=args.nsamples,
        noise=args.noise,
        centers=args.centers,
        cluster_std=args.clusterstd,
        val_split=args.valsplit,
        test_split=args.testsplit,
        seed=0,
    )
    hidden = [int(x) for x in args.hidden_sizes.split(",") if x.strip()]
    train_cfg = TrainConfig(
        run_dir=args.rundir,
        hidden_sizes=hidden,
        lr=args.lr,
        weight_decay=args.weightdecay,
        epochs=args.epochs,
        batch_size=args.batchsize,
        device=args.device,
    )
    seeds = [int(s) for s in args.seeds.split(",") if s.strip()]
    benchmark(data_cfg, train_cfg, seeds)


# ------------------------------
# Main (Notebook/Colab-safe)
# ------------------------------

def main(argv: Optional[List[str]] = None):
    import sys

    raw_args = argv if argv is not None else sys.argv[1:]
    filtered_args = sanitize_argv(raw_args)

    parser = build_parser()

    # Early exit in notebooks/empty arg cases
    if not filtered_args:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--run_dir", "runs/moonstest"])')
        print('main(["eval", "--run_dir", "runs/moonstest"])')
        print('main(["export-onnx", "--run_dir", "runs/moonstest"])')
        return

    try:
        args = parser.parse_args(filtered_args)
    except SystemExit:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--run_dir", "runs/moonstest"])')
        print('main(["eval", "--run_dir", "runs/moonstest"])')
        print('main(["export-onnx", "--run_dir", "runs/moonstest"])')
        return

    cmd = getattr(args, "command", None)
    if cmd == "train":
        cmd_train(args)
    elif cmd == "eval":
        cmd_eval(args)
    elif cmd == "predict":
        cmd_predict(args)
    elif cmd == "export-onnx":
        cmd_export_onnx(args)
    elif cmd == "benchmark":
        cmd_benchmark(args)
    else:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--run_dir", "runs/moonstest"])')
        print('main(["eval", "--run_dir", "runs/moonstest"])')
        print('main(["export-onnx", "--run_dir", "runs/moonstest"])')


if __name__ == "__main__":
    main()